|E-MMR-W| |A|B|C|D|E|
|:----|:----|:----|:----|:----|:----|:----|
|N=6|P|0.470375|0.519828|0.501495|0.537972|0.518142|
| |Q|0.760864|0.764936|0.765095|0.771853|0.772681|
| |R|0.621411|0.675804|0.659599|0.697599|0.672377|
| |F|0.604833|0.655057|0.640695|0.674588|0.653912|
| | | | | | | |
|MMR-E| |A|B|C|D|E|
|N=6|P|0.4541|0.44845|0.4763|0.50223|0.48693|
| |Q|0.97147|0.93344|0.95565|0.96601|0.95805|
| |R|0.4656|0.47498|0.49479|0.51269|0.50499|
| |F|0.59927|0.6001|0.62674|0.66316|0.63632|
| | | | | | | |
|文字置き換え| |A|B|C|D|E|
|N=6|P|0.50505|0.55656|0.54214|0.57634|0.55197|
| |Q|0.81003|0.81033|0.81323|0.80784|0.80577|
| |R|0.61599|0.67422|0.65981|0.69921|0.67193|
| |F|0.63972|0.69362|0.68036|0.71194|0.68787|
| | | | | | | |
|AMRバージョン| |A|B|C|D|E|
|N=6|P|0.56875|0.59712|0.59444|0.65688|0.6118|
| |Q|0.75981|0.75095|0.75146|0.75411|0.75162|
| |R|0.72279|0.77884|0.77234|0.83173|0.78664|
| |F|0.70124|0.72862|0.72796|0.77593|0.74093|

In [13]:
# import Pkg
# Pkg.add("DataFrames")
# Pkg.add("CSV")
# Pkg.add("IntervalSets")
# Pkg.add("LatexStrings")

In [14]:
using DataFrames, CSV
using IntervalArithmetic
using LaTeXStrings
using Statistics
# using Base
include("./libs/display-latex.jl")
include("./libs/crisp-pcm.jl")
include("./libs/analysis-indicators.jl")
include("./libs/solve-deterministic-ahp.jl")
include("./libs/MMR-kai.jl")

MMR_kai (generic function with 1 method)

In [15]:
# 評価基準数
num_criteria = [6];
# 真の区間重要度のセッティング
true_importance = ["A", "B", "C", "D", "E"]; # ["A", "B", "C", "D", "E"]

In [16]:
# 真の区間重要度を読み込み
for N in num_criteria
        
    for setting in true_importance
        df = CSV.File("./data/N"*string(N)*"_"*setting*"_Given_interval_weight.csv", header=false) |> DataFrame
        T = Vector{Interval{Float64}}() # Float64のところにTを使うとおかしくなるのはなぜか？
        for i in 1:N
            push!(T, interval(df[1, 2*i-1], df[1, 2*i]))
        end

        # display(L"T = %$(intervalVectorLaTeXString(T))")
        # 一対比較行列を1000個読み込み
        df1 = CSV.File("./data/N"*string(N)*"_a3_"*setting*"_PCM_int.csv", header=false) |> DataFrame

        # データフレームをN×Nのサブデータフレームに分割
        subdfs = split_dataframe(df1, N);
        # 結果格納用
        P = Matrix{Float64}(undef, N, 1000)
        Q = Matrix{Float64}(undef, N, 1000)
        R = Matrix{Float64}(undef, N, 1000)
        F = Matrix{Float64}(undef, N, 1000)

        # 1000個のデータで推定
        for (i, subdf) in enumerate(subdfs)

            # 一対比較行列を取り出す
            A = Matrix(subdf)
            # display(L"A = %$(matrixLaTeXString(A))")
            # if i == 779
            #     println("i = ", i)
            #     display(L"A = %$(matrixLaTeXString(A))")
            # end

            # 提案手法で区間重要度推定
            solution = MMR_kai(A, EV)
            E = solution.W # 区間重要度の推定値
            # display(L"E = %$(intervalVectorLaTeXString(E))")
            # display(L"T = %$(intervalVectorLaTeXString(T))")

            TcapE = T .∩ E # 真の区間重要度と推定値の積集合
            TcupE = T .∪ E # 真の区間重要度と推定値の和集合
            # display(L"T ∩ E = %$(intervalVectorLaTeXString(TcapE))")
            # display(L"T ∪ E = %$(intervalVectorLaTeXString(TcupE))")
            
            # P, Q, R, F の計算
            P[:, i] = calculate_P(T, E) # 真の区間重要度と推定値の積集合の割合
            Q[:, i] = calculate_Q(T, E) # 真の区間重要度と推定値の積集合の割合
            R[:, i] = calculate_R(T, E) # 真の区間重要度と推定値の積集合の割合
            F[:, i] = calculate_F(T, E) # F値
            # println("i = ", i)
            # display(L"P = %$(VectorLaTeXString(P[:,i]))")
            # display(L"Q = %$(VectorLaTeXString(Q[:,i]))")
            # display(L"R = %$(VectorLaTeXString(R[:,i]))")
            # display(L"F = %$(VectorLaTeXString(F[:,i]))")
        end

        println("N = ", N)
        println("setting = ", setting)

        P_ave = mean(P)
        Q_ave = mean(Q)
        R_ave = mean(R)
        F_ave = mean(F)

        display(L"Pₐᵥₑ = %$(round(P_ave, digits=5))")
        display(L"Qₐᵥₑ = %$(round(Q_ave, digits=5))")
        display(L"Rₐᵥₑ = %$(round(R_ave, digits=5))")
        display(L"Fₐᵥₑ = %$(round(F_ave, digits=5))")
        # P_var = var(P)
        # Q_var = var(Q)
        # R_var = var(R)
        # F_var = var(F)
        # display(L"Pᵥₐᵣ = %$(round(P_var, digits=5))")
        # display(L"Qᵥₐᵣ = %$(round(Q_var, digits=5))")
        # display(L"Rᵥₐᵣ = %$(round(R_var, digits=5))")
        # display(L"Fᵥₐᵣ = %$(round(F_var, digits=5))")

    end
end


L"$Pₐᵥₑ = 0.50505$"

L"$Qₐᵥₑ = 0.81003$"

L"$Rₐᵥₑ = 0.61599$"

L"$Fₐᵥₑ = 0.63972$"

N = 6
setting = A


...

L"$Pₐᵥₑ = 0.55197$"

L"$Qₐᵥₑ = 0.80577$"

L"$Rₐᵥₑ = 0.67193$"

L"$Fₐᵥₑ = 0.68787$"

N = 6
setting = E
